In [2]:
import pandas as pd

In [ ]:
# input: n_people (e.g. 1000)
# determine number of people in each age bin - 
from utils import relativefreq2absolutefreq
age_histogram = relativefreq2absolutefreq(
    bins_fractions={age_bin: p for age_bin, p in conf.get('HUMAN_DISTRIBUTION').items()},
    n_elements=n_people,
    rng=rng
)


In [24]:
# https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/prof/details/page.cfm?Lang=E&Geo1=HR&Code1=2406&Geo2=PR&Code2=01&SearchText=montreal&SearchType=Contains&SearchPR=01&B1=Families,%20households%20and%20marital%20status&TABID=1&type=0

# Region
# Région de Montréal Quebec

# Household and Dwelling characteristic
# household size distribution
# fraction of **all** households that are 1 person households 0.394 (A.1)
# fraction of **all** households that are 2 persons households 0.298 (A.2)
# fraction of **all** households that are 3 persons households 0.136 (A.3)
# fraction of **all** households that are 4 persons households 0.112 (A.4)
# fraction of **all** households that are more than 5 persons households 0.06 (A.5)
# average household size 2.2
P_HOUSEHOLD_SIZE = [0.394, 0.298, 0.136, 0.112, 0.06]

# Household type (B)
# total private households - 870,375 (B.1)
# one census-family households - 469,260 (0.539) (B.2)

# Family characteristics (C)
# age-household size distribution
# total at least one census family households 485,285 (C.1)
# 2 person families 
#   2 person families in private households - 241,425 (C.2)
#   fraction of **all** households that are 2 person families = (C.2 / C.1) * (B.2 / B.1) = 0.268 (C.2.a)
#   fraction of **all** households that are of size 2 and not 2 person families = (C.2.a - A.2) = 0.03
#       couples without children - 178,110 (C.2.1) 
#       fraction of **all** households that are "couples without children" = (C.2.1 / C.1) * (B.2 / B.1) = 0.198
#       single parent with a child - 63,315 (C.2.2)
#       fraction of **all** households that are "single parent with a child" - (C.2.2 / C.1) * (B.2 / B.1) = 0.070
P_FAMILY_TYPE_SIZE_2 = [0.198, 0.070, 0.03]

# 3 person families 
#   3 person families in private households - 109,085 (C.3)
#   fraction of **all** households that are 3 person families = (C.3 / C.1) * (B.2 / B.1) = 0.121 (C.3.a)
#   fraction of **all** households that are of size 3 and not 3 person families = (C.3.a - A.3) = 0.015
#       couples with 1 child - 82,310 (C.3.1)
#       fraction of **all** households that are "couples with 1 child" = (C.3.1 / C.1) * (B.2 / B.1) = 0.091      
#       single parent with 2 children - 26,780 (C.3.2)
#       fraction of **all** households that are "single parent with 2 children" = (C.3.2 / C.1) * (B.2 / B.1) = 0.03
P_FAMILY_TYPE_SIZE_3 = [0.091, 0.03, 0.015]

# 4 person families
#   4 person families in private households - 93,225 (C.4)
#   fraction of **all** households that are 4 person families = (C.4 / C.1) * (B.2 / B.1) = 0.104 (C.4.a)
#   fraction of **all** households that are of size 4 and not 4 person families = (C.4.a - A.4) = 0.008 (C.4.b)
#       couples with 2 children - 85,985 (C.4.1)
#       fraction of **all** households that are "couples with 2 children" = (C.4.1 / C.1) * (B.2 / B.1) = 0.096 (C.4.1.a)  
#       single parent with 3 or more children - 9,610 (C.4.2)
#       single parent with 3 children - 9,610 (C.4 - C.4.1) = 7240 (C.4.3)
#       fraction of **all** households that are "single parent with 3 children" = (C.4.3 / C.1) * (B.2 / B.1) = 0.008
P_FAMILY_TYPE_SIZE_4 = [0.096, 0.008, 0.008]

# 5 or more person families 
#   5 person families in private households - 41,555 (C.5)
#   fraction of **all** households that are 5 person families = (C.5 / C.1) * (B.2 / B.1) = 0.046 (C.5.a)
#   fraction of **all** households that are of size more than 5 and not more than 5 person families = (C.5.a - A.5) = 0.014
#      couples with 3 or more children - 39,180 (C.5.1)
#      fraction of **all** households that are "couples with 3 or more children" = (C.5.1 / C.1) * (B.2 / B.1) = 0.044
#      single parent with 4 or more children - (C.4.2 - C.4.3) = 2370 (C.5.2)
#      fraction of **all** households that are "single parent with 4 or more children" = (C.5.2 / C.1) * (B.2 / B.1) = 0.003
P_FAMILY_TYPE_SIZE_MORE_THAN_5 = [0.044, 0.003, 0.014]

# multi-generation households (D)
# https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/hlt-fst/fam/Table.cfm?Lang=E&T=24&Geo=00&SO=18D
# Total private households  - 779,800 (D.1)
# multi-generation households - 11,410 (D.2)
# fraction of **all** households that are multigenerational - (D.2 / D.1)  = 0.015
# (assumed) multi-generation households are one of more than 3 persons families. 
# These are sampled when family_type = other.
# Minimum age sampled for grandparent = 45
# ref: https://www150.statcan.gc.ca/n1/pub/75-006-x/2015001/article/14154-eng.htm
P_MULTIGENERATIONAL_FAMILY = 0.015

# Senior residency & long term care centers
# ref: https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/prof/details/page.cfm?Lang=E&Geo1=HR&Code1=2406&Geo2=PR&Code2=01&SearchText=montreal&SearchType=Contains&SearchPR=01&B1=All&TABID=1&type=0
# total population = 1,942,045 (E)
# ref: https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/covid19/table2-eng.cfm?geo=A0002&S=1&O=A
# total population living in collectives for seniors - 32935 (E.1)
# total population aged 0 to 79 living in collectives for seniors - 9750 (E.2) 
# total population aged 80 or above living in collectives for seniors - 23185 (E.3)
# probability of living in a senior residency given 0 < age < 79 (E.2 / E)
# NOTE: (assumption) minimum age for collective is 70 yo
P_COLLECTIVE_70_79 = 0.005
# (assumption) equal distribution for two age bins
P_COLLECTIVE_70_74 = 0.0025
P_COLLECTIVE_75_79 = 0.0025
# probability of living in a senior residency given age > 80
P_COLLECTIVE_80_above = 0.017

# Solo dwellers 
P_AGE_SOLO_DWELLERS_GIVEN_HOUSESIZE_1 = []
# procedure - 
# Data for solo dwellers (across Canada) is given here
# # Data for chart 1: https://www150.statcan.gc.ca/n1/pub/75-006-x/2019001/article/00003-eng.htm
# project it back per age group as per Montreal's demographics
# Multiply by a correction term to make number of solo houses equal to A.1 * TOTAL_HOUSES
# Normalize it to get probabilities given housesize of 1

In [175]:
# https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/prof/details/page.cfm?Lang=E&Geo1=HR&Code1=2406&Geo2=PR&Code2=01&SearchText=montreal&SearchType=Contains&SearchPR=01&B1=All&TABID=1&type=1
POPULATION_SIZE_REGION = 1942045
P_AGE = [
    [0, 4, 0.057],
    [5, 9, 0.054],
    [10, 14, 0.047],
    [15, 19, 0.05],
    [20, 24, 0.072],
    [25, 29, 0.079],
    [30, 34, 0.078],
    [35, 39, 0.076],
    [40, 44, 0.066],
    [45, 49, 0.065],
    [50, 54, 0.068],
    [55, 59, 0.065],
    [60, 64, 0.057],
    [65, 69, 0.049],
    [70, 74, 0.037],
    [75, 79, 0.029],
    [80, 110, 0.051],
]

# https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/prof/details/page.cfm?Lang=E&Geo1=HR&Code1=2406&Geo2=PR&Code2=01&SearchText=montreal&SearchType=Contains&SearchPR=01&B1=All&TABID=1&type=1
POPULATION_SIZE_CANADA = 35151730
P_AGE_CANADA = [
    [0, 4, 0.054],
    [5, 9, 0.057],
    [10, 14, 0.055],
    [15, 19, 0.058],
    [20, 24, 0.064],
    [25, 29, 0.065],
    [30, 34, 0.066],
    [35, 39, 0.065],
    [40, 44, 0.064],
    [45, 49, 0.067],
    [50, 54, 0.076],
    [55, 59, 0.075],
    [60, 64, 0.065],
    [65, 69, 0.056],
    [70, 74, 0.04],
    [75, 79, 0.029],
    [80, 110, 0.044],
]
# print(sum(x[2] for x in P_AGE_CANADA)) ; 0.999
# NOTE: sum of probabilities = 0.999. As a result, p for [80,110] has been increased by 0.001


In [176]:
# Data for chart 1: https://www150.statcan.gc.ca/n1/pub/75-006-x/2019001/article/00003-eng.htm
with open("data/solo_dwellers_age.txt","r") as f:
    data = f.read()

In [177]:
age_bins = [(x[0], x[1]) for x in P_AGE]
N_AGE_SOLO_DWELLERS_CANADA = [[x[0], x[1], 0] for x in age_bins]
for line in data.split("\n")[2:]:
    line = line.split("\t")
    if line[0] == "100 and over":
        age = 100
    else:
        age = int(line[0])
    total = int(line[2]) + int(line[4])
    for i,x in enumerate(age_bins):
        if x[0] <= age <= x[1]:
            break
    N_AGE_SOLO_DWELLERS_CANADA[i][-1] += total

In [179]:
N_AGE_SOLO_DWELLERS = [[x[0], x[1], 0] for x in age_bins]
idx = 0
for mtl, canada, solo in zip(P_AGE, P_AGE_CANADA, N_AGE_SOLO_DWELLERS_CANADA):
    N_AGE_SOLO_DWELLERS[idx][-1] = (solo[2] / (canada[2] * POPULATION_SIZE_CANADA)) * mtl[2] * POPULATION_SIZE_REGION
    idx += 1
    
total_solo_dwellers = sum(x[2] for x in N_AGE_SOLO_DWELLERS)

In [180]:
# ref: https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/prof/details/page.cfm?Lang=E&Geo1=HR&Code1=2406&Geo2=PR&Code2=01&SearchText=montreal&SearchType=Contains&SearchPR=01&B1=All&TABID=1&type=0
# Total number of 1 person private households in Montreal - 342,510
# normalize
N_HOUSESIZE_1 = 342510
correction_factor  = N_HOUSESIZE_1 / total_solo_dwellers
N_AGE_SOLO_DWELLERS_CORRECTED = [[x[0], x[1], x[2] * correction_factor] for x in N_AGE_SOLO_DWELLERS]

In [181]:
P_AGE_SOLO_DWELLERS_GIVEN_HOUSESIZE_1 = [[x[0], x[1], x[2]/N_HOUSESIZE_1] for x in N_AGE_SOLO_DWELLERS_CORRECTED]

In [182]:
P_AGE_SOLO_DWELLERS_GIVEN_HOUSESIZE_1

[[0, 4, 0.0],
 [5, 9, 0.0],
 [10, 14, 0.0],
 [15, 19, 0.0036965443759304443],
 [20, 24, 0.04050656352664703],
 [25, 29, 0.08236366540850498],
 [30, 34, 0.08036315583876404],
 [35, 39, 0.06662188307122593],
 [40, 44, 0.05447753487688209],
 [45, 49, 0.061166114151859505],
 [50, 54, 0.07899697367438215],
 [55, 59, 0.08849240507820566],
 [60, 64, 0.08991688613071544],
 [65, 69, 0.08616853066878354],
 [70, 74, 0.07314565348534838],
 [75, 79, 0.0648654340868155],
 [80, 110, 0.12921865562593535]]

In [183]:
M

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16
0,0.592072,0.437310,0.200605,0.084645,0.129332,0.291811,0.551087,0.530414,0.194535,0.065874,0.054157,0.027575,0.020712,0.007347,0.002825,0.005444
1,0.271954,0.849879,0.371932,0.116641,0.031965,0.139007,0.430889,0.575518,0.419683,0.123987,0.040448,0.021569,0.010694,0.007253,0.002873,0.003316
2,0.124213,0.376400,1.359936,0.353181,0.047379,0.027110,0.129980,0.384804,0.517711,0.224737,0.066414,0.016871,0.009984,0.008610,0.006602,0.003508
3,0.058301,0.114780,0.418480,1.233644,0.183147,0.035972,0.029132,0.181945,0.388922,0.432658,0.195403,0.047837,0.014416,0.011297,0.004810,0.002686
4,0.123387,0.056612,0.076434,0.387866,1.285891,0.218355,0.047626,0.021713,0.136269,0.424249,0.257116,0.111626,0.018516,0.003938,0.004179,0.003365
5,0.396486,0.137085,0.034841,0.069130,0.237530,1.112928,0.225381,0.032172,0.015585,0.069264,0.193781,0.112456,0.042852,0.008593,0.001127,0.005784
6,0.569773,0.528767,0.219452,0.038818,0.055205,0.208882,1.020880,0.221001,0.073919,0.017837,0.034256,0.056715,0.055306,0.008412,0.004017,0.003439
7,0.465074,0.711659,0.544765,0.196201,0.023614,0.029065,0.151100,0.998726,0.171510,0.036781,0.021204,0.014379,0.027006,0.014676,0.006522,0.002285
8,0.186802,0.457012,0.646953,0.417653,0.089305,0.022337,0.070274,0.170996,0.811102,0.136315,0.032235,0.005240,0.018483,0.020605,0.008303,0.005255
9,0.107774,0.239847,0.436497,0.642947,0.345140,0.070086,0.026160,0.080566,0.169446,0.845174,0.145544,0.029301,0.010896,0.008479,0.007277,0.013141


In [96]:
# home contact matrix
M = pd.read_csv("data/canada-contact-matrix.csv")
contact_matrix_age_bins = [[0,4], [5,9], [10,14], [15,19], [20,24], [25,29], [30,34], [35,39], [40,44], \
                         [45,49], [50,54], [55,59], [60,64], [65,69], [70,74], [75, 110]]

In [ ]:
def _get_demographics_adjusted_to_contact_matrix_agebins(P_AGE, POPULATION_SIZE):
    N = []
    for i, x in enumerate(P_AGE):
        if x[1] <= 29:
            N += [[x[0], x[1], x[2] * POPULATION_SIZE]]
        elif x[0]%10 == 0 and x[1] < 70:
            # break the buckets into two age groups
            N += [[x[0], x[0] + 4, x[2] * 0.5 * POPULATION_SIZE]]
            N += [[x[0] + 5, x[1], x[2] * 0.5 * POPULATION_SIZE]]
        elif x[0] == 70:
            # aggregate the age groups above 70
            N += [[x[0], x[0] + 4, x[2] * 0.5 * POPULATION_SIZE]]
            total_proportion = sum(x[2] for x in P_AGE[i+1:]) + x[2] * 0.5
            N += [[x[0] + 5, 110, total_proportion * POPULATION_SIZE]] # 75+ yo
            break
        else:
            raise

    assert len(N) == len(contact_matrix_age_bins), "age bins of contact matrix do not align with internal age bins"
    assert abs(sum(x[2] for x in N) - POPULATION_SIZE) < 1e-02, f"populations do not sum up, {sum(x[2] for x in N)} != {POPULATION_SIZE}"
    
    return N

In [ ]:
N_CANADA_adjusted = _get_demographics_adjusted_to_contact_matrix_agebins(P_AGE_CANADA, POPULATION_SIZE_CANADA)
N_REGION_adjusted = _get_demographics_adjusted_to_contact_matrix_agebins(P_AGE, POPULATION_SIZE_REGION)

In [97]:
# until empirical estimates are available, we use locaiton specific contact matrices derived in Prem et al. 
# We project the matrix for Canada to Montreal to account for demographics changes
# NOTE: Data from Prem et al. has 16 classes while original POLYMOD study has 15 classes with last class as 70+ yo
# (assumption until verified) Prem et al. has broken down the last class as 70-74 and 75+ yo resulting in an extra class

# for the definition of M_ij look at "Epidemiological Modelling: Simulating the Initial Phase of an Epidemic" of the original POLYMOD study
# ref: https://journals.plos.org/plosmedicine/article?id=10.1371/journal.pmed.0050074#s2
# M_ij - average number of daily contacts reported by an individual in age group j with an individual in estimated age group i


In [171]:
# deriving population contact matrix C from raw contact matrix M

# adjust for local population structure using "Projecting social contact matrices to different demographic structures"
# ref: https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1006638#sec018
# method M2 Density correction 
# N_j = Number of individuals in age group j in the reference country
# N = population size of the reference country
# N` =  population size of the concerned region
# N_j` = Number of individuals in age group j in the concerned region
# M_ij` = M_ij * (N / N_j) * (N_j` / N`)

M_density_corrected = pd.DataFrame(index=M.index, columns=M.columns)
for j in range(len(M.columns)):
    N_j = N_CANADA_adjusted[j][2]
    N_j_dash = N_REGION_adjusted[j][2]
    N = POPULATION_SIZE_CANADA
    N_dash = POPULATION_SIZE_REGION
    M_density_corrected.iloc[:,j] = M.iloc[:,j] * (N / N_j) * (N_j_dash / N_dash)

# adjust for reciprocity
# following the procedure described in BBC pandemic study 
# ref: https://www.medrxiv.org/content/10.1101/2020.02.16.20023754v2.full.pdf
# C_ij = (M_ij + M_ji *(w_i / w_j)) * 0.5
C = pd.DataFrame(index=M.index, columns=M.columns)
for j in range(len(M.columns)):
    for i in range(len(M.columns)):
        w_i = N_REGION_adjusted[i][2]
        w_j = N_REGION_adjusted[j][2]
        C.iloc[i,j] = 0.5 * (M_density_corrected.iloc[i, j] + M_density_corrected.iloc[j, i] * w_i / w_j)

# NOTE: C is not symmetric matrix. Reciprocity translates into number of contacts from i to j to be equal to number of contacts
# from j to i. However, C is an average matrix so the numbers will not be symmetric. 

# convert the contact matrix into likelihood of interaction between different age groups
P = pd.DataFrame(index=C.index, columns=C.columns)
for j in range(len(C.columns)):
    P.iloc[:, j] = C.iloc[:, j] / C.iloc[:, j].sum()

assert all(abs(P.sum(axis=0) - 1) < 1e-2)